# Creacion Consolidado por ROL

Comenzaremos el analisis de la data consolidando la info en un unico DataSet, para ello vamos a considerar los siguientes puntos:


- DATA:  Se considerará toda las causas que ingresaron al Sistema desde 2015 al 2019, esto ya que se cuenta además con la data de las audiencias en ese período.


- No se consideran causas anteriores a este período.

- Se considerarán Causas sin termino.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from pjud.data import cleandata
from pjud.data import transformdata

In [2]:
warnings.filterwarnings(action="ignore")
tqdm.pandas()

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
path_processed = "../data/processed/pjud"

#### Paso 1 : Consolidacion de Dataset x rol

In [5]:
df_ingresos_rol = pd.read_feather(f"{path_processed}/processes_IngresosRol.feather")
print(f"{len(df_ingresos_rol)} causas en el dataset de ingresos por Rol")

2978550 causas en el dataset de ingresos por Rol


In [6]:
df_termino_rol = pd.read_feather(f"{path_processed}/processes_TerminosRol.feather")
print(f"Existen : {len(df_termino_rol)} causas en el dataset de termino por rol")

Existen : 2944068 causas en el dataset de termino por rol


Analizamos la data, uniendo ambos dataset.

In [7]:
df_fulldata_rol = pd.merge(df_ingresos_rol, df_termino_rol, how='outer', on=['COD. TRIBUNAL','RIT'])
print(f"Existen: {len(df_fulldata_rol)} causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso")

Existen: 3322673 causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso


In [8]:
df_fulldata_rol['AÑO INGRESO'].value_counts()

2018.0    672071
2015.0    669056
2016.0    665737
2017.0    660914
2019.0    652535
Name: AÑO INGRESO, dtype: int64

### Analisis de la Data Causas por Rol

De estos dos dataset debo considerar:

- Caso 1: el dataset df_causas_terminadas completo
- Caso 2: Debo agregar además las causas que tengan Fecha Ingreso_x pero que aún no terminan.
- Caso 3: Debo analizar las causas que no tienen Fecha Ingreso_x pero si tienen Fecha Ingreso_y y Fecha Ingreso_y, ya que deben ser las causas que no fueron tomadas por la consulta de Ingresos. Acá se debe reemplazar los datos obtenidos en Termino Rol a Ingresos.

Luego de tener estos tres dataset debo concatenarlos

In [9]:
df_fulldata_rol.sample(10)

,index_x,COD. CORTE_x,CORTE_x,COD. TRIBUNAL,TRIBUNAL_x,RIT,TIPO CAUSA_x,FECHA INGRESO_x,MES INGRESO,AÑO INGRESO,index_y,COD. CORTE_y,CORTE_y,TRIBUNAL_y,TIPO CAUSA_y,FECHA INGRESO_y,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
1267519,566589.0,91.0,C.A. DE SAN MIGUEL,1231,DUODECIMO JUZGADO DE GARANTIA SANTIAGO,941-2019,Ordinaria,2019-02-20,feb-19,2019.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
874590,192871.0,35.0,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA RANCAGUA,2579-2019,Ordinaria,2019-03-11,mar-19,2019.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
849121,168582.0,35.0,C.A. DE RANCAGUA,114,JUZGADO DE LETRAS Y GARANTIA PEUMO,691-2019,Ordinaria,2019-05-08,may-19,2019.0,182201.0,35.0,C.A. DE RANCAGUA,JUZGADO DE LETRAS Y GARANTIA PEUMO,Ordinaria,2019-05-08,2019-05-20,12.0,ACUMULACION.,may-19,2019.0,1.0
1187450,490680.0,90.0,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,5981-2019,Ordinaria,2019-04-10,abr-19,2019.0,531736.0,90.0,C.A. DE SANTIAGO,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2019-04-10,2019-04-10,0.0,DECLARA INCOMPETENCIA.,abr-19,2019.0,1.0
287133,259372.0,46.0,C.A. DE CONCEPCION,1075,JUZGADO DE GARANTIA SAN PEDRO DE LA PAZ,335-2018,Ordinaria,2018-02-28,feb-18,2018.0,294652.0,46.0,C.A. DE CONCEPCION,JUZGADO DE GARANTIA SAN PEDRO DE LA PAZ,Ordinaria,2018-02-28,2019-02-08,345.0,CERTIFICA CUMPLIMIENTO ART. 468.,feb-19,2019.0,1.0
1761789,379314.0,90.0,C.A. DE SANTIAGO,1221,SEGUNDO JUZGADO DE GARANTIA SANTIAGO,4576-2015,Ordinaria,2015-04-08,abr-15,2015.0,438864.0,90.0,C.A. DE SANTIAGO,SEGUNDO JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2015-04-08,2015-04-08,0.0,APROBACION NO INICIO INVESTIGACION.,abr-15,2015.0,1.0
3254632,524087.0,91.0,C.A. DE SAN MIGUEL,1230,UNDECIMO JUZGADO DE GARANTIA SANTIAGO,10539-2016,Ordinaria,2016-12-29,dic-16,2016.0,592992.0,91.0,C.A. DE SAN MIGUEL,UNDECIMO JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2016-12-29,2016-12-29,0.0,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,dic-16,2016.0,1.0
2479171,426959.0,90.0,C.A. DE SANTIAGO,1225,SEXTO JUZGADO DE GARANTIA SANTIAGO,5046-2017,Ordinaria,2017-07-05,jul-17,2017.0,477142.0,90.0,C.A. DE SANTIAGO,SEXTO JUZGADO DE GARANTIA SANTIAGO,Ordinaria,2017-07-05,2017-07-05,0.0,APROBACION NO INICIO INVESTIGACION.,jul-17,2017.0,1.0
2956531,264150.0,46.0,C.A. DE CONCEPCION,1082,JUZGADO DE GARANTIA CONCEPCION,113-2016,Ordinaria,2016-01-07,ene-16,2016.0,296970.0,46.0,C.A. DE CONCEPCION,JUZGADO DE GARANTIA CONCEPCION,Ordinaria,2016-01-07,2016-01-08,1.0,CERTIFICA CUMPLIMIENTO ART. 468.,ene-16,2016.0,1.0
2902328,216740.0,40.0,C.A. DE TALCA,967,JUZGADO DE GARANTIA LINARES,2948-2016,Ordinaria,2016-09-09,sept-16,2016.0,242502.0,40.0,C.A. DE TALCA,JUZGADO DE GARANTIA LINARES,Ordinaria,2016-09-09,2016-09-12,3.0,ACOGE REQUERIMIENTO (MONITORIO).,sept-16,2016.0,1.0


In [10]:
df_fulldata_rol.columns

Index(['index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL', 'TRIBUNAL_x',
       'RIT', 'TIPO CAUSA_x', 'FECHA INGRESO_x', 'MES INGRESO', 'AÑO INGRESO',
       'index_y', 'COD. CORTE_y', 'CORTE_y', 'TRIBUNAL_y', 'TIPO CAUSA_y',
       'FECHA INGRESO_y', 'FECHA TERMINO', 'DURACION CAUSA', 'MOTIVO TERMINO',
       'MES TERMINO', 'AÑO TERMINO', 'TOTAL TERMINOS'],
      dtype='object')

In [11]:
columnas_drop = ['index_x', 'index_y', 'MES INGRESO', 'MES TERMINO']

df_fulldata_rol.drop(columnas_drop, axis = 'columns', inplace = True)

In [12]:
df_fulldata_rol = df_fulldata_rol.progress_apply(transformdata.faltantes_rol, axis=1)

100%|██████████| 3322673/3322673 [06:41<00:00, 8268.55it/s] 


In [13]:
columnas_drop = ['TIPO CAUSA_y', 'TRIBUNAL_y', 'COD. CORTE_y', 'CORTE_y', 'FECHA INGRESO_y']

df_fulldata_rol.drop(columnas_drop, axis = 'columns', inplace = True)

In [14]:
df_fulldata_rol.rename(columns = {'COD. CORTE_x':'COD. CORTE',
                                  'CORTE_x':'CORTE',
                                  'TRIBUNAL_x':'TRIBUNAL',
                                  'TIPO CAUSA_x':'TIPO CAUSA',
                                  'MATERIA_x':'MATERIA',
                                  'FECHA INGRESO_x':'FECHA INGRESO'
                                  }, inplace = True)

In [15]:
df_fulldata_rol.sample(10)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
1355497,11.0,C.A. DE IQUIQUE,993,JUZGADO DE GARANTIA IQUIQUE,13728-2015,Ordinaria,2015-12-18,2015.0,2015-12-21,3.0,APROBACION NO INICIO INVESTIGACION,2015.0,1.0
2803445,30.0,C.A. DE VALPARAISO,1051,JUZGADO DE GARANTIA LA LIGUA,127-2016,Ordinaria,2016-01-28,2016.0,2016-01-28,0.0,ACOGE REQUERIMIENTO (MONITORIO),2016.0,1.0
688884,11.0,C.A. DE IQUIQUE,989,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,875-2019,Ordinaria,2019-12-04,2019.0,NaT,NaN,SIN TERMINO,NaN,NaN
3088286,90.0,C.A. DE SANTIAGO,1221,SEGUNDO JUZGADO DE GARANTIA SANTIAGO,1621-2016,Ordinaria,2016-02-10,2016.0,2016-02-10,0.0,APROBACION NO INICIO INVESTIGACION,2016.0,1.0
3176940,90.0,C.A. DE SANTIAGO,1227,OCTAVO JUZGADO DE GARANTIA SANTIAGO,2723-2016,Ordinaria,2016-04-04,2016.0,2017-08-07,490.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2017.0,1.0
3204355,90.0,C.A. DE SANTIAGO,1232,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,2236-2016,Ordinaria,2016-03-30,2016.0,2017-07-26,483.0,SENTENCIA,2017.0,1.0
1101790,60.0,C.A. DE COYHAIQUE,994,JUZGADO DE GARANTIA COIHAIQUE,3090-2019,Ordinaria,2019-10-04,2019.0,2019-10-04,0.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,2019.0,1.0
2899178,40.0,C.A. DE TALCA,967,JUZGADO DE GARANTIA LINARES,231-2016,Ordinaria,2016-01-20,2016.0,2016-04-22,93.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2016.0,1.0
2432758,90.0,C.A. DE SANTIAGO,1221,SEGUNDO JUZGADO DE GARANTIA SANTIAGO,5915-2017,Ordinaria,2017-06-02,2017.0,2017-06-02,0.0,SENTENCIA,2017.0,1.0
496282,90.0,C.A. DE SANTIAGO,1225,SEXTO JUZGADO DE GARANTIA SANTIAGO,4827-2018,Ordinaria,2018-06-21,2018.0,2018-06-21,0.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,2018.0,1.0


In [16]:
df_fulldata_rol['AÑO INGRESO'].value_counts()

2018.0    672490
2015.0    669645
2016.0    666291
2017.0    661446
2019.0    652801
Name: AÑO INGRESO, dtype: int64

In [17]:
df_fulldata_rol['MOTIVO TERMINO'].value_counts()

DECLARA SOBRESEIMIENTO DEFINITIVO                   704733
APROBACION NO INICIO INVESTIGACION                  636026
SENTENCIA                                           518385
SIN TERMINO                                         378602
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD    357975
ACOGE REQUERIMIENTO (MONITORIO)                     311540
NO PERSEVERAR EN EL PROCEDIMIENTO                   178922
DECLARA INCOMPETENCIA                                87256
CERTIFICA CUMPLIMIENTO ART 468                       66816
ACUMULACION                                          46394
DECLARA INCOMPETENCIA RPA                            19918
DECLARA INADMISIBILIDAD DE LA QUERELLA                9106
ABANDONO DE LA QUERELLA                               5281
DESISTIMIENTO QUERELLA                                1719
Name: MOTIVO TERMINO, dtype: int64

In [18]:
print(f"Existen: {len(df_fulldata_rol)} causas por rol")

Existen: 3322673 causas por rol


In [19]:
causas_top = df_fulldata_rol[df_fulldata_rol['TRIBUNAL'].str.contains('ORAL')]
causas_garantia = df_fulldata_rol[df_fulldata_rol['TRIBUNAL'].str.contains('GARANTIA')]


In [20]:
causas_top.sample(10)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
672452,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,437-2019,Ordinaria,2019-12-04,2019.0,NaT,NaN,SIN TERMINO,NaN,NaN
3319564,91.0,C.A. DE SAN MIGUEL,1322,TRIBUNAL DE JUICIO ORAL EN LO PENAL PUENTE ALTO,42-2016,Ordinaria,2016-05-03,2016.0,2016-10-12,162.0,DECLARA INCOMPETENCIA,2016.0,1.0
1708291,55.0,C.A. DE VALDIVIA,1251,TRIBUNAL DE JUICIO ORAL EN LO PENAL OSORNO,69-2015,Ordinaria,2015-05-20,2015.0,2015-06-19,30.0,SENTENCIA,2015.0,1.0
2793630,30.0,C.A. DE VALPARAISO,1048,TRIBUNAL DE JUICIO ORAL EN LO PENAL VIÑA DEL MAR,383-2016,Ordinaria,2016-10-26,2016.0,2016-12-02,37.0,SENTENCIA,2016.0,1.0
584112,90.0,C.A. DE SANTIAGO,1247,CUARTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN...,433-2018,Ordinaria,2018-08-31,2018.0,2018-10-24,54.0,SENTENCIA,2018.0,1.0
1480989,30.0,C.A. DE VALPARAISO,1054,TRIBUNAL DE JUICIO ORAL EN LO PENAL VALPARAISO,347-2015,Ordinaria,2015-07-08,2015.0,2015-08-26,49.0,SENTENCIA,2015.0,1.0
1245978,90.0,C.A. DE SANTIAGO,1321,TRIBUNAL DE JUICIO ORAL EN LO PENAL COLINA,109-2019,Ordinaria,2019-08-20,2019.0,2019-10-07,48.0,SENTENCIA,2019.0,1.0
1244041,90.0,C.A. DE SANTIAGO,1245,SEGUNDO TRIBUNAL DE JUICIO ORAL EN LO PENAL SA...,355-2019,Ordinaria,2019-07-11,2019.0,2019-12-23,165.0,SENTENCIA,2019.0,1.0
1910295,90.0,C.A. DE SANTIAGO,1246,TERCER TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN...,272-2015,Ordinaria,2015-11-26,2015.0,2016-01-19,54.0,SENTENCIA,2016.0,1.0
584072,90.0,C.A. DE SANTIAGO,1247,CUARTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN...,395-2018,Ordinaria,2018-08-10,2018.0,2018-09-28,49.0,SENTENCIA,2018.0,1.0


In [21]:
causas_top.query("RIT == '309-2019' and `COD. TRIBUNAL`== 1323")

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
1324285,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,309-2019,Ordinaria,2019-09-04,2019.0,2019-12-07,94.0,SENTENCIA,2019.0,1.0


In [22]:
causas_garantia.sample(10)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
1647360,46.0,C.A. DE CONCEPCION,189,JUZGADO DE LETRAS Y GARANTIA SANTA JUANA,287-2015,Ordinaria,2015-09-01,2015.0,2015-09-02,1.0,APROBACION NO INICIO INVESTIGACION,2015.0,1.0
1666875,50.0,C.A. DE TEMUCO,937,JUZGADO DE GARANTIA TEMUCO,5417-2015,Ordinaria,2015-05-26,2015.0,2015-05-26,0.0,APROBACION NO INICIO INVESTIGACION,2015.0,1.0
288172,46.0,C.A. DE CONCEPCION,1075,JUZGADO DE GARANTIA SAN PEDRO DE LA PAZ,1245-2018,Ordinaria,2018-07-06,2018.0,2018-07-06,0.0,ACOGE REQUERIMIENTO (MONITORIO),2018.0,1.0
804757,30.0,C.A. DE VALPARAISO,1046,JUZGADO DE GARANTIA VIÑA DEL MAR,8016-2019,Ordinaria,2019-06-26,2019.0,2019-08-27,62.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2019.0,1.0
1592805,45.0,C.A. DE CHILLAN,152,JUZGADO DE LETRAS Y GARANTIA QUIRIHUE,544-2015,Ordinaria,2015-08-26,2015.0,2015-08-26,0.0,SENTENCIA,2015.0,1.0
2935798,46.0,C.A. DE CONCEPCION,1075,JUZGADO DE GARANTIA SAN PEDRO DE LA PAZ,2471-2016,Ordinaria,2016-12-02,2016.0,2016-12-16,14.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2016.0,1.0
2033819,15.0,C.A. DE ANTOFAGASTA,951,JUZGADO DE GARANTIA CALAMA,6685-2017,Ordinaria,2017-10-20,2017.0,2018-02-23,126.0,SENTENCIA,2018.0,1.0
1804343,90.0,C.A. DE SANTIAGO,1224,QUINTO JUZGADO DE GARANTIA SANTIAGO,1805-2015,Ordinaria,2015-04-21,2015.0,2015-05-14,23.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2015.0,1.0
1975622,91.0,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA SAN BERNARDO,10362-2015,Ordinaria,2015-10-19,2015.0,2015-12-03,45.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2015.0,1.0
313318,46.0,C.A. DE CONCEPCION,1082,JUZGADO DE GARANTIA CONCEPCION,3309-2018,Ordinaria,2018-04-15,2018.0,2019-06-03,414.0,DECLARA SOBRESEIMIENTO DEFINITIVO,2019.0,1.0


In [23]:
causas_top['MOTIVO TERMINO'].value_counts()

SENTENCIA                            61904
SIN TERMINO                           4583
DECLARA SOBRESEIMIENTO DEFINITIVO     1654
ACUMULACION                            638
DECLARA INCOMPETENCIA                  203
ABANDONO DE LA QUERELLA                 44
DECLARA INCOMPETENCIA RPA                3
Name: MOTIVO TERMINO, dtype: int64

In [24]:
causas_garantia['MOTIVO TERMINO'].value_counts()

DECLARA SOBRESEIMIENTO DEFINITIVO                   703079
APROBACION NO INICIO INVESTIGACION                  636026
SENTENCIA                                           456481
SIN TERMINO                                         374019
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD    357975
ACOGE REQUERIMIENTO (MONITORIO)                     311540
NO PERSEVERAR EN EL PROCEDIMIENTO                   178922
DECLARA INCOMPETENCIA                                87053
CERTIFICA CUMPLIMIENTO ART 468                       66816
ACUMULACION                                          45756
DECLARA INCOMPETENCIA RPA                            19915
DECLARA INADMISIBILIDAD DE LA QUERELLA                9106
ABANDONO DE LA QUERELLA                               5237
DESISTIMIENTO QUERELLA                                1719
Name: MOTIVO TERMINO, dtype: int64

ATENCION:
Se detecta que existen RIT 0-AÑO, lo cual no existe en la realidad, estos RIT deberían corresponder a errores del Sistema o ajustes para Metas de Gestión. Para efecto de este analisis deberían ELIMINARSE.

In [25]:
df_rit_cero = df_fulldata_rol[df_fulldata_rol['RIT'].str.startswith("0-")]

In [26]:
df_rit_cero

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,AÑO INGRESO,FECHA TERMINO,DURACION CAUSA,MOTIVO TERMINO,AÑO TERMINO,TOTAL TERMINOS
3320314,11.0,C.A. DE IQUIQUE,6,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,0-2018,Ordinaria,2018-09-13,2018.0,2018-09-12,1.0,DECLARA INCOMPETENCIA,2018.0,1.0
3320315,11.0,C.A. DE IQUIQUE,6,JUZGADO DE LETRAS Y GARANTIA POZO ALMONTE,0-2018,Ordinaria,2018-09-13,2018.0,2018-12-31,109.0,DECLARA INCOMPETENCIA,2018.0,1.0
3320327,15.0,C.A. DE ANTOFAGASTA,951,JUZGADO DE GARANTIA CALAMA,0-2018,Ordinaria,2018-02-19,2018.0,2018-02-05,14.0,DECLARA INCOMPETENCIA,2018.0,1.0
3320328,15.0,C.A. DE ANTOFAGASTA,951,JUZGADO DE GARANTIA CALAMA,0-2018,Ordinaria,2018-02-19,2018.0,2018-12-27,311.0,DECLARA INCOMPETENCIA,2018.0,1.0
3320338,25.0,C.A. DE LA SERENA,930,JUZGADO DE GARANTIA COQUIMBO,0-2018,Ordinaria,2018-04-28,2018.0,2018-11-13,199.0,DECLARA INCOMPETENCIA,2018.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3322549,90.0,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-12-02,2015.0,2015-12-03,1.0,DECLARA INCOMPETENCIA,2015.0,1.0
3322550,90.0,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-12-22,2015.0,2015-12-31,9.0,DECLARA INCOMPETENCIA,2015.0,1.0
3322585,90.0,C.A. DE SANTIAGO,1232,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-10-04,2015.0,2015-10-14,10.0,CERTIFICA CUMPLIMIENTO ART 468,2015.0,1.0
3322586,90.0,C.A. DE SANTIAGO,1232,DECIMOTERCER JUZGADO DE GARANTIA SANTIAGO,0-2015,Ordinaria,2015-11-10,2015.0,2015-12-21,41.0,DECLARA INCOMPETENCIA,2015.0,1.0


In [27]:
df_fulldata_rol.drop(df_rit_cero.index, axis=0, inplace=True)

## CARGA DATA A ARCHIVOS FUTHER

In [28]:
# Directorio donde se guardaran archivos feather
df_fulldata_rol.reset_index(inplace = True)
causas_top.reset_index(inplace = True)
causas_garantia.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldata_rol.to_feather(f'{path_processed}/consolidated_Rol.feather')
causas_top.to_feather(f'{path_processed}/consolidated_JuicioOralesRol.feather')
causas_garantia.to_feather(f'{path_processed}/consolidated_CausasGarantiaRol.feather')